In [2]:
from utils.extract_pdf_contents import process_pdfs
from utils.clean_texts import clean_EU_legal_text

## Research paper NLP Pipeline

#### 1. Extract the text from the two pdf documents

In [ ]:
# Define the input and output directories
input_directory = "data/raw"  # Directory containing the PDF files
output_directory = "data/extracted_text"  # Directory to save the extracted text files


# Call the function to process the PDFs
process_pdfs(input_directory, output_directory)

Extracted text saved to: data/extracted_text/EU_AI_Act_English.txt
Extracted text saved to: data/extracted_text/USA_AI_Executive_Order_English.txt


#### 2. Cleanup the extracted text

In [3]:
# First the EU Act

# Read the file content and pass it to clean_legal_text
with open("data/extracted_text/EU_AI_Act_English.txt", "r", encoding="utf-8") as file:
    text = file.read()

clean_text = clean_EU_legal_text(text)

# Save cleaned text to /data/cleaned_text directory
with open("data/cleaned_text/EU_AI_Act_English_Cleaned.txt", "w", encoding="utf-8") as file:
    file.write(clean_text)


# Second, the USA AI Executive Order

#### 3. Tokenize the text

#### 4. Text representation via BoW and TF-IDF

#### 4.2. Text representation through Word2Vec emdeddings

#### 4.3. Text representation through OpenAI contextualised embeddings